In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf
import datetime
from datetime import date

yf.pdr_override()

In [5]:
pd.set_option('display.max_rows', 10)

df = pd.read_csv('TICKERS.csv')

df.columns = ['empresa','ticker1','ticker2','ticker3']

df = df.melt(id_vars=['empresa'])

df.drop(columns=['variable'],inplace=True)

df.dropna(inplace=True)

df.set_index('empresa',inplace=True)

df.sort_index(key=lambda col: col.str.lower(),inplace=True)



In [46]:
aux = []

#Carregar os dados

for i in range(df.shape[0]):
    aux.append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start='2021-01-01',end = '2021-05-04'));
    
df.insert(1, 'dados', aux, allow_duplicates=True);

print('======= ACABOU ===========')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ALPK3.SA: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[************

In [96]:
#nan_value = float("NaN")
#Convert NaN values to empty string

#df.replace("", nan_value, inplace=True)

#df.dropna(subset = ["dados"], inplace=True)

C:\Users\hugo_\anaconda3\lib\site-packages\pandas\core\missing.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [10]:
# salvar data frame

df.to_hdf('b3test.h5', 'obj1');

In [7]:
# carregar data frame
df = pd.DataFrame();
aux = [];
df = pd.read_hdf('b3test.h5', 'obj1');
for i in range(df.shape[0]):
    aux.append(df.iloc[i,1])

In [4]:
# atualização

from datetime import datetime

data = datetime.fromisoformat('1970-01-01')

for i in range(df.shape[0]):
    if (df.iloc[i,1].shape[0]>0):
        if (max(df.iloc[i,1].index) > data):
            data = max(df.iloc[i,1].index)

for i in range(df.shape[0]):
    df2 = df.iloc[i,1].append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start=data))
    df2 = df2[~df2.index.duplicated(keep='first')]
    df.iat[i,1] = df2

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
#setup 9.1
#calculo da MME9
tend = pd.DataFrame();
for i in range(df.shape[0]):
    
    aux[i]['MME9'] = aux[i].Close.ewm(span=9).mean().dropna()
    aux[i]["SLOPE"]="";
    for k in range (aux[i].shape[0]):
        tend = aux[i].MME9[k] - aux[i].MME9[k-1]
        aux[i].iat[k,7]=tend    
                
#inicializar o 1 valor de slope como 0
#for i in range(df.shape[0]):
   # if(aux[i].SLOPE[0] == True):
       # aux[i].iat[0,7]=0
#print('don')                     
            

In [18]:
#for i in range(df.shape[0]):
#    del aux[i]["SLOPE"]

In [9]:
#perca de minima
for i in range(df.shape[0]):
   aux[i]['mark_min'] = np.where((aux[i].Close < aux[i].shift(1).Low) & (aux[i].SLOPE > 0) & (aux[i].index > '2021-04-26'),1,0)

In [23]:
#saber quantos mark_min
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_min[j] !=0):
            contador = contador + 1;
contador

# Achar os mark_min
# aux[0].loc[aux[0]['mark_min'] == 1]

45

In [10]:
#superação de maxima
for i in range(df.shape[0]):
   aux[i]['mark_max'] = np.where(((aux[i].Close > aux[i].shift(1).High) | (aux[i].Open > aux[i].shift(1).High)) & (aux[i].SLOPE > 0) & ((aux[i].shift(1).mark_min == 1) | (aux[i].shift(2).mark_min == 1) | (aux[i].shift(3).mark_min == 1)) ,1,0)


In [25]:
#saber quantos mark_max
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_max[j] !=0):
            contador = contador + 1;
contador

17

In [11]:
#pegar todos os tickers com mark_max e as datas
tickersmax = [];
datamax = [];
tickersmin = [];
datamin = [];
for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if (aux[i]['mark_max'][j] == 1):
            tickersmax.append(df.iloc[i,0]);
            datamax.append(aux[i].index[j]);
        if (aux[i]['mark_min'][j] == 1):
            tickersmin.append(df.iloc[i,0]);
            datamin.append(aux[i].index[j]);
#alocar os resultados em dois pd            
df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
#tirar valores duplicados ficando com o mais recente

df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
#setup 9.2 armado          
df_setup_92_armado = pd.DataFrame();
duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]

In [60]:
#declarar coluna vazia slope
#for i in range(df.shape[0]):
    #aux[i]["SLOPE"]="";


In [81]:
for i in range (teste.shape[0]):
    if teste.iloc[i,1] == duplicates:
        print(teste.iloc[i,1])

In [140]:
df

,value,dados
empresa,,
3M,MMMC34,Open High Low...
3R Petroleum,RRRP3,Open High Low ...
Abbott Laboratories,ABTT34,Open High Low...
Aeris,AERI3,Open High Low Close Adj Cl...
AES Brasil,AESB3,Open High Low ...
...,...,...
Whirlpool,WHRL4,Open High Low Close Adj Close...
Whirpool,WHRL3,Open High Low Close Adj Close...
Xerox,XRXB34,Open High Low...


In [18]:
#watchlist
intradayaux = []
del df_setup_92_armado['dados']
for i in range(df_setup_92_armado.shape[0]):
    #startdate = df_setup_92_armado.iloc[i,0];
    today = date.today();
    tomorrow = date.today() + datetime.timedelta(days=1)
    intradayaux.append(web.get_data_yahoo(df_setup_92_armado.iloc[i,1]+'.sa',start= today,end = tomorrow,interval="5m"));
    
df_setup_92_armado.insert(2, 'dados', intradayaux, allow_duplicates=True);

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [193]:
if intradayaux[i].Close[intradayaux[i].shape[0]-1] >= 

In [17]:
intradayaux[0].Close[intradayaux[0].shape[0]-1]

18.790000915527344